In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

CHIP_TABLE = pd.read_csv('./data/ChIP_NARROW.csv')
CHIP_TABLE

### Download SRA

In [9]:
import sys, subprocess
OUTDIR= '/LocalFastqDir/...'
NT = '12'
RANGE=(0,32)

#
for i,row in CHIP_TABLE.iterrows():
    if i in range(RANGE[0],RANGE[1]):
        SRA = row['run_accession']
        SAMPLE_ID = row['SAMPLE_ID']
        print(f'\nIndex     = {i}')
        print(f'SRA       = {SRA}')
        print(f'SAMPLE_ID = {SAMPLE_ID}')
        print(f'OUTDIR    = {OUTDIR}\n')
        subprocess.run(['parallel-fastq-dump','--tmpdir','.','--sra-id',SRA,'--threads',NT,'--outdir',OUTDIR,'--gzip'])

### Map with bowtie2 (~25 min x sample)

In [7]:
MAPFILES='/LocalMapDir/...'
SSD_DIR='/StorageVolume/...'

for i,row in CHIP_TABLE.iterrows():
    if i in range(RANGE[0],RANGE[1]):
        try:
            SRA = row['run_accession']
            SAMPLE_ID = row['SAMPLE_ID']
            print(f'\nIndex     = {i}')
            print(f'SRA       = {SRA}')
            print(f'SAMPLE_ID = {SAMPLE_ID}')
            print(f'OUTDIR    = {OUTDIR}\n')
            #subprocess.run(['parallel-fastq-dump','--tmpdir','.','--sra-id',SRA,'--threads',NT,'--outdir',OUTDIR,'--gzip'])
            FASTQ_FILE = OUTDIR+row['run_accession']+'.fastq.gz'
            print(f'\nIndex      = {i}')
            print(f'FASTQ_FILE = {FASTQ_FILE}')
            print(f'SAMPLE_ID  = {SAMPLE_ID}\n')
            print(f'{MAPFILES}{SAMPLE_ID}.bam\n')
            #subprocess.run(['pigz', '-d', '-k', '-p','5', FASTQ_FILE+'.gz'])
            subprocess.run(['./MapChIPseq2.pl', '-v', 'mouse', 'mm10', FASTQ_FILE, SAMPLE_ID])
            subprocess.run(['rm', '-f', FASTQ_FILE])
            subprocess.run(['rsync', '-a', '-v','-h','--remove-source-files', f'{MAPFILES}{SAMPLE_ID}.bam', SSD_DIR])
        except Exception:
            print('Error')
            break

### Assert all the SAMPLE IDs have a BAM file


In [36]:
import os
SAMPLE_IDS = pd.Index([sample_id + '.bam' for sample_id in CHIP_TABLE['SAMPLE_ID']])
BAMS = pd.Index(os.listdir(SSD_DIR))
assert len(SAMPLE_IDS.intersection(BAMS)) == len(SAMPLE_IDS)